In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import make_pipeline

In [ ]:
PATH = "../input/tabular-playground-series-jan-2021/"
train = pd.read_csv(f"{PATH}train.csv",index_col='id')
test = pd.read_csv(f"{PATH}test.csv",index_col='id')
submission= pd.read_csv(f"{PATH}sample_submission.csv",index_col='id')

# Removing outliers


In [ ]:
from scipy import stats
z = np.abs(stats.zscore(train))
train = train[(z < 3).all(axis=1)]

# Base Models

In [ ]:
train_c = train.copy()

In [ ]:
X = train_c.iloc[:,:-1].values
y = train_c.iloc[:,14].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10,random_state=0)

# Model Stacking

****

# XGBoost Regressor


In [ ]:
model_xgb = xgb.XGBRegressor()

model_xgb.fit(X_train, y_train, early_stopping_rounds=7, eval_set=[(X_test, y_test)], verbose=1)



In [ ]:
model_xgb_pred = model_xgb.predict(X_test)
score_reg = mean_squared_error(y_test, model_xgb_pred, squared=False)
xgb_pred=model_xgb.predict(test.values)
print(f'{score_reg:0.5f}')

# LGBM Regressor


In [ ]:
model_lgb = lgb.LGBMRegressor()
model_lgb.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)], verbose=1)

In [ ]:
model_lgb_pred =model_lgb.predict(X_test)
score_reg = mean_squared_error(y_test, model_lgb_pred, squared=False)
lgb_pred=(model_lgb.predict(test.values))
print(f'{score_reg:0.5f}')

In [ ]:
model_lgb.feature_importances_

In [ ]:
def rmsle(y, y_pred):
    mean_squared_error(y, y_pred,squared=False)
    return print(f'{score_reg:0.5f}')

In [ ]:
print('RMSLE score on train data:')
print(mean_squared_error(y_test, 0.75*model_lgb_pred+0.25*model_xgb_pred, squared=False))

In [ ]:
xgb_pred=model_xgb.predict(test.values)
lgb_pred=model_lgb.predict(test.values)

In [ ]:
ensemble = 0.75*lgb_pred+0.25*xgb_pred

In [ ]:
submission['target'] = ensemble
submission.to_csv('model_lgb_xgb.csv')